In [1]:
import pandas as pd
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler

pd.options.display.max_columns = None

train = pd.read_csv("../datasources/houseprices/train.csv")
test = pd.read_csv("../datasources/houseprices/test.csv")
full = pd.concat([train, test]) # Assemble les deux jeux de données

Informations statistiques (sur les variables numériques)

In [2]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Nombre de lignes / colonnes

In [3]:
train.shape

(1460, 81)

Informations / colonnes (type et NaN)

In [41]:
train.info(verbose=True,null_counts =True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [38]:
train.get_dtype_counts()

float64     3
int64      35
object     43
dtype: int64

In [72]:
print("Nombre de colonnes: " + str(train.columns.shape[0]))
print("Nombre de lignes: " + str(train.shape[0]))
print("Affichage des colonnes ayant des données NaN:")
for col in train.columns:
    nanCount = train[col].isnull().sum()
    if (nanCount > 0):
        print("   * " + col + "> " + str(nanCount) + " (" + str(round(nanCount / train.shape[0] * 100, 2)) + " %)")
print("Affichage des colonnes Non numériques:")
for col in train.columns:
    myType = str(train[col].dtype)
    if (myType[:3] != "int" and myType[:5] != "float"):
        print("   * " + col + "> " + myType)

Nombre de colonnes: 81
Nombre de lignes: 1460
Affichage des colonnes ayant des données NaN:
   * LotFrontage> 259 (17.74 %)
   * Alley> 1369 (93.77 %)
   * MasVnrType> 8 (0.55 %)
   * MasVnrArea> 8 (0.55 %)
   * BsmtQual> 37 (2.53 %)
   * BsmtCond> 37 (2.53 %)
   * BsmtExposure> 38 (2.6 %)
   * BsmtFinType1> 37 (2.53 %)
   * BsmtFinType2> 38 (2.6 %)
   * Electrical> 1 (0.07 %)
   * FireplaceQu> 690 (47.26 %)
   * GarageType> 81 (5.55 %)
   * GarageYrBlt> 81 (5.55 %)
   * GarageFinish> 81 (5.55 %)
   * GarageQual> 81 (5.55 %)
   * GarageCond> 81 (5.55 %)
   * PoolQC> 1453 (99.52 %)
   * Fence> 1179 (80.75 %)
   * MiscFeature> 1406 (96.3 %)
Affichage des colonnes Non numériques:
   * MSZoning> object
   * Street> object
   * Alley> object
   * LotShape> object
   * LandContour> object
   * Utilities> object
   * LotConfig> object
   * LandSlope> object
   * Neighborhood> object
   * Condition1> object
   * Condition2> object
   * BldgType> object
   * HouseStyle> object
   * RoofStyle> o

In [91]:
for col in train.columns:
    print("   * " + col + "\n")
    print(str(train[col].value_counts(sort=True).head(5)) + "\n")

   * Id

1460    1
479     1
481     1
482     1
483     1
Name: Id, dtype: int64

   * MSSubClass

20     536
60     299
50     144
120     87
30      69
Name: MSSubClass, dtype: int64

   * MSZoning

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

   * LotFrontage

60.0    143
70.0     70
80.0     69
50.0     57
75.0     53
Name: LotFrontage, dtype: int64

   * LotArea

7200     25
9600     24
6000     17
10800    14
9000     14
Name: LotArea, dtype: int64

   * Street

Pave    1454
Grvl       6
Name: Street, dtype: int64

   * Alley

Grvl    50
Pave    41
Name: Alley, dtype: int64

   * LotShape

Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

   * LandContour

Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64

   * Utilities

AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64

   * LotConfig

Inside     1052
Corner      263
CulDSac      94
FR2          47
F

In [86]:
print('col> ' + str(train['SaleCondition'].value_counts().head(10)))

col> Normal     1198
Partial     125
Abnorml     101
Family       20
Alloca       12
AdjLand       4
Name: SaleCondition, dtype: int64
